In [1]:
import numpy as np
from scipy import ndimage
import skimage.feature as feature
import matplotlib.pyplot as plt
import time
import cv2

In [2]:
def gaussian_kernel(h, std_dev):
    # It calculates and Returns a normalized Gaussian kernel with dimension (h x h)
    kernel = np.zeros((h, h))
    center = h // 2
    
    for i in range(h):
        for j in range(h):
            x, y = i - center, j - center
            kernel[i][j] = np.exp(-(x**2 + y**2) / (2 * std_dev**2))
    return kernel / np.sum(kernel)

In [3]:
def gradient_image(img):
    #It calculates the gradient images of x and y using the Sobel filter and Gaussian smoothing.   
    # Defining the Sobel filters
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    
    # Applying Gaussian smoothing with sigma = 0.2
    gaussian_kernel_3x3 = gaussian_kernel(5, 0.2)
    img_smoothed = ndimage.convolve(img, gaussian_kernel_3x3, mode='reflect')
    
    # Calculating the gradient images through Sobel filters
    Ix = ndimage.convolve(img_smoothed, sobel_x, mode='reflect')
    Iy = ndimage.convolve(img_smoothed, sobel_y, mode='reflect')
    
    return Ix, Iy

In [4]:
def harris_response(Ix, Iy, w, alpha=0.01):
    #It calculates the Harris corner response R for all pixels in the image.
    # Calculating the IxIx, IyIy and IxIy matrices
    IxIx = ndimage.convolve(Ix**2, w, mode='reflect')
    IyIy = ndimage.convolve(Iy**2, w, mode='reflect')
    IxIy = ndimage.convolve(Ix * Iy, w, mode='reflect')
    
    # Calculating the Harris response R
    det = IxIx * IyIy - IxIy**2
    trace = IxIx + IyIy
    R = det - alpha * trace**2
    
    return R

In [5]:
def response_non_maxsup(R, img_shape):
    corners = feature.peak.peak_local_max(R, min_distance=7, threshold_rel=0.01)
    return corners

In [6]:
def extract_patches(img, keypoints, N=32):
    #First we create border, then create blob of N pixels, according to assignment, N should be 32.
    patches = []
    img = cv2.copyMakeBorder(img, top=N//2, bottom=N//2, left=N//2, right=N//2, borderType=cv2.BORDER_REPLICATE)
    for x, y in keypoints:
        x += N//2
        y += N//2
        patch = img[y-N//2:y+N//2, x-N//2:x+N//2]
        patches.append(patch)
    return patches


In [7]:
def extract_normalized_patches(patches):
    # It calculates the normalized patches by applying the farmula available on the assignment document
    normalized_patches = []
    for patch in patches:
        normalized_patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-7)
        normalized_patches.append(normalized_patch)
    return normalized_patches


In [8]:
def distance_matrix(normalized_patches1, normalized_patches2):
    K1 = len(normalized_patches1)
    K2 = len(normalized_patches2)
    N = normalized_patches1[0].shape[0]   
    distance_matrix = np.zeros((K1, K2))
    for i in range(K1):
        for j in range(K2):
            patch1 = normalized_patches1[i]
            patch2 = normalized_patches2[j]
            if patch2.shape[0] == patch1.shape[0] and patch2.shape[1] == patch1.shape[1]  :
                distance_matrix[i, j] = np.sum((patch1 - patch2)**2)

    return distance_matrix

In [9]:
def find_matches(D):
    #It calculates the matches 
    matches = []
    for i in range(D.shape[1]):
        index = np.argmin(D[:, i])
        if index != i:
            matches.append((index, i))
    return matches


In [10]:
def find_matches_robust(patches1, patches2):
    #It calculates the matches by applying ratio test
    D = distance_matrix(patches1, patches2)
    matched_indexes = []
    for i in range(D.shape[1]):
        column = D[:,i]
        nn1_index = np.argmin(column)
        nn2_index = np.argsort(column)[1]
        nn1_distance = column[nn1_index]
        nn2_distance = column[nn2_index]
        ratio = nn1_distance / nn2_distance
        if ratio < 0.8:
            matched_indexes.append((nn1_index, i))
    filtered_matches = []
    for index1, index2 in matched_indexes:
        patch1 = patches1[index1]
        patch2 = patches2[index2]
        if np.linalg.norm(patch1 - patch2) < 0.1:
            filtered_matches.append((index1, index2))
    return filtered_matches

In [11]:
def average_vs_detected_keypoints(matches):
    #It calculates average number of detected and tracked keypoints
    keypoints_per_frame = []
    for i in range(199):
        keypoints_per_frame.append(len(matches))
    average_keypoints = sum(keypoints_per_frame) / len(keypoints_per_frame)
    return average_keypoints

In [12]:
def extract_sift_descriptors(corners, image, patch_size=32):
    img = cv2.merge([image, image, image]).astype(np.uint8)
    sift_descriptors = []
    sift = cv2.SIFT_create()
    for corner in corners:
        x, y = corner
        patch = img[x-patch_size//2:x+patch_size//2, y-patch_size//2:y+patch_size//2]
        #if image.depth() != cv2.CV_8U:
        _,descriptors = sift.compute(patch, None)
        sift_descriptors.append(descriptors)
    return sift_descriptors

In [13]:
def make_video ( list_image_names ) :
    # reads the images
    img_array = []
    for filename in sorted ( list_image_names ):
        img = cv2.imread ( filename )
        img_array.append ( img )

    # video size = image size
    height , width , layers = img . shape
    size = ( width , height )
    # create mp4 writer with 3 FPS
    video_wr = cv2.VideoWriter ('corner_tracking_patch .mp4', cv2.VideoWriter_fourcc (* 'mp4v') , 3 , size )
    for i in range(len(img_array)):
        video_wr.write(img_array [i])
        video_wr.release ()

In [14]:
def plot_matches(image_tgt, list_keypoints_src, list_keypoints_tgt, matches):
    image_color = cv2.cvtColor(image_tgt, cv2.COLOR_GRAY2BGR)
    # add cross for each keypoint in target image
    for keyp in list_keypoints_tgt:
        cv2.drawMarker(image_color, (keyp[1], keyp[0]), (0, 0, 255),
        markerType = cv2.MARKER_CROSS, markerSize = 3, thickness = 3, 
        line_type = cv2.LINE_8)
    # draw green lines of matches
    for i, j in matches:
        coord_src, coord_tgt = list_keypoints_src[i], list_keypoints_tgt[j]
    # please note that coordinates are inverted to follow opencv convention (x, y) instead of (y, x)
        cv2.line(image_color,(coord_tgt[1], coord_tgt[0]),(coord_src[1], coord_src[0]),(0, 255, 0),1)
    # display image
    cv2.imshow("Matches", image_color)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [15]:
def plot_corners(img, corners):
    plt.imshow(img, cmap='gray')
    plt.plot(corners[:, 1], corners[:, 0], 'r+', markersize=10, markeredgewidth=2)
    plt.show()

In [ ]:
# test on the frames
for i in range (1,80):
    star1= str(i-1).zfill(6)
    path1 = "images/" + star1 + ".png"
    img1 = plt.imread("images/000000.png")
    #img = np.mean(img, axis=2)
    Ix1, Iy1 = gradient_image(img1)
    R1 = harris_response(Ix1, Iy1, gaussian_kernel(5, 1))
    corners1 = response_non_maxsup(R1, img1.shape)
    patches1= extract_patches(img1, corners1, 32)
    normalized_patches1= extract_normalized_patches(patches1)

    star2= str(i).zfill(6)
    path2 = "images/" + star2 + ".png"
    img2 = plt.imread("images/000000.png")
    #img = np.mean(img, axis=2)
    Ix2, Iy2 = gradient_image(img2)
    R2 = harris_response(Ix2, Iy2, gaussian_kernel(5, 1))
    corners2 = response_non_maxsup(R2, img2.shape)
    patches2= extract_patches(img2, corners2, 32)
    normalized_patches2= extract_normalized_patches(patches2)

    #normalized_patches1 = extract_sift_descriptors(corners1,img1)
    #normalized_patches2 = extract_sift_descriptors(corners2 , img2)

    #for patch in normalized_patches2:
    #    print("The patch is", patch.shape[0]) 

    Distance=distance_matrix(normalized_patches2, normalized_patches1)
    Matches= find_matches(Distance)
    Robust_matches= find_matches_robust(normalized_patches1, normalized_patches2)
    #Comparison = average_vs_detected_keypoints (Matches)
    plot_matches (img2, corners1, corners2, Matches)
    #plot_corners(img1, corners1)
    time.sleep(1)